<a href="https://colab.research.google.com/github/IamArmanNikkhah/ParsParaphraser/blob/main/parsparaphraser_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Dataset Ready ⚓

**Download The Dataset**

In [ ]:
! pip install gdown

In [ ]:
!gdown --id 150_bdU7auDRp0SFvcrPJUwqardySNDYA   # IF an error occured try again!

In [ ]:
import pandas as pd
df = pd.read_csv('./para-nmt-train.csv')  # CAUTION!! :  edit it kaggle

**Split Dataset**

In [ ]:
def split_dataframe(df, n):
    # Get the number of rows in the Dataframe
    rows = df.shape[0]
    # Get the number of rows in each part
    part_rows = int(rows / n)
    # Create a list to store the Dataframes
    list_df = []
    # Start the loop
    for i in range(n):
        # Create the start and end rows
        start_row = i * part_rows
        end_row = (i + 1) * part_rows
        # Create the Dataframe
        df_part = df.iloc[start_row:end_row]
        # Add the Dataframe to the list
        list_df.append(df_part)
    # Return the list of Dataframes
    return list_df

In [ ]:
list_df = split_dataframe(df,8)

❗You should change this part :

**⏩split_num : the split number that you should translate**

In [ ]:
split_num = 5               # CAUTION!! :  change it in accordance with the instruction
df        = list_df[split_num]

# Translating

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
#translator = Translator()

def translate(text):
    try:
      translator = Translator()
      translation = translator.translate(text, dest='fa')
    except:
        return text
    else:
        return translation.text

In [ ]:
import multiprocessing as mp
import numpy as np

def apply_func(df, func=translate):
    df.iloc[:,0] = df.iloc[:,0].apply(func)
    df.iloc[:,1] = df.iloc[:,1].apply(func)
    return df

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = mp.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def chunk_df(df, n):
    """
    Chunk a dataframe into n equal parts.
    """
    return [df.iloc[i::n, :] for i in range(n)]

num_partitions = 4              #number of partitions to split dataframe
num_cores      = 4              #number of cores on your machine
chunk_num      = 1000

filename       = "Translated_" + str(split_num) + ".csv"                 # Don't Change IT !!
directory      = "./"                                                    # Change it if needed and add / at the end

print(f"Translating Process has been started on the chunk number :{split_num}\n")
for chunk in chunk_df(df, chunk_num):
    chunk_translated = parallelize_dataframe(chunk, apply_func)
    chunk_translated.to_csv(directory + filename, mode='a')